In this notebook:
    - pull USGS data for length that each gage is in operation
    - potentially cut time scale of data based on when the flow has never been above a certain threshold
    - each of the target gages should have a dataframe (list of dataframes)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import time

In [2]:
import pickle
path="C:\Springboard\Github\gauge_info"
os.chdir(path)

In [3]:
# load the dataframe of targets and their corresponding features
dt = pickle.load(open("USGS_targets.pkl", "rb"))
dt

,USGS,lat,long,alt,basin,lng,feat_USGS,feat_NOAA
0,10140700,41.231819,111.984497,4285.00,Lower Weber,-111.984497,"[10126000, 10092700, 10010000, 10171000, 10105...","[BCNU1, BIUI1, GSPU1, JRSU1, PRZU1, LCJU1, LGN..."
1,10149000,40.118012,111.314622,6320.00,Spanish Fork,-111.314622,"[10164500, 10133800, 10133650, 10133600, 09313...","[AFPU1, ECAU1, ECPU1, MCLU1, PRHU1, PVHU1, SCJ..."
2,10155200,40.554398,111.433243,5691.59,Provo,-111.433243,"[10164500, 10131000, 10134500, 10132500, 10155...","[AFPU1, CIVU1, ECCU1, CRDU1, PVHU1, RBCU1, CLL..."
79,09064600,39.553875,106.402529,8078.37,Eagle,-106.402529,"[09057500, 09034250, 09034500, 09065100, 09063...","[BGMC2, CAWC2, HTSC2, CSSC2, RERC2, FRGC2, FPT..."
84,09081000,39.373317,107.083937,6470.00,Roaring Fork,-107.083937,"[09132095, 09070500, 09085100, 09081600, 09070...","[ACSC2, EGLC2, GCOC2, RCYC2, GPSC2, ENMC2, GEP..."
85,09073300,39.141100,106.774204,8120.00,Roaring Fork,-106.774204,"[09065100, 09112500, 09078600, 09080400, 09066...","[CSSC2, ALEC2, FPTC2, RUDC2, GRVC2, GUSC2, HUN..."
86,09075400,39.193833,106.833667,7882.00,Roaring Fork,-106.833667,"[09065100, 09112500, 09078600, 09080400, 09066...","[CSSC2, ALEC2, FPTC2, RUDC2, GRVC2, GUSC2, OHO..."
87,09033300,40.006892,105.848272,8274.00,Colorado Headwaters,-105.848272,"[09050700, 09010500, 09034250, 09019500, 09034...","[BLRC2, BAKC2, CAWC2, CBGC2, HTSC2, FRGC2, FRW..."
88,09359010,37.802774,107.672839,9245.98,Animas,-107.672839,"[09126000, 09118450, 09165000, 09074000, 09365...","[CMRC2, CRCC2, DRRC2, HUNC2, LPHC2, LFBC2, LFG..."
92,09358000,37.811108,107.659228,9290.00,Animas,-107.659228,"[09126000, 09118450, 09165000, 09074000, 09365...","[CMRC2, CRCC2, DRRC2, HUNC2, LPHC2, LFBC2, LFG..."


That looks good. Now what? :) <br>

How do I structure the data that I pull for each of these gages? <br>

I think we may need to pull these files individually and save them to a CSV. Then for the modeling, we load the CSV's that we need. Otherwise, it's too much memory.

In [4]:
# function for pulling the data and saving it to CSV
# takes in string of the gage number with USGS
# def pull_water(USGS):
USGS = '10140700'
link = "https://waterdata.usgs.gov/nwis/dv?referred_module=sw&site_no=" + USGS
response = requests.get(link)
soup = BeautifulSoup(response.text, "html.parser")


# find td

# go to next td after that
# format should be YEAR-MONTH-DATE
# extract that string (using RegEx); stop at '&'
# start_date = extracted date

# do another web scrape for the actual date
# just save that to CSV - we can parse it in the next notebook




# then use start date to get CSV of data
# link: https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=10140700&referred_module=sw&period=&begin_date=2012-04-11&end_date=2021-05-28
# https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=10140700&referred_module=sw&period=&begin_date=<year-mon-day>&end_date=2021-05-28
        

In [5]:
# inside id="available_parameters_table"
site_sum = soup.find_all(id='available_parameters_table')
print(len(site_sum))

1


In [6]:
# look inside the rows
rows_sum = site_sum[0].find_all('tr')
print(len(rows_sum))

3


In [7]:
print(rows_sum[2])

<tr>
<td width="5%"><font face="Verdana,Arial,Helvetica" size="-1"><input alt="Check for Parameter Code" checked="checked" class="available_parameters" id="checkbox_00060" name="cb_00060" ts_ids="ts_ids_143565" type="checkbox"/></font></td>
<td><label for="checkbox_00060"><font face="Verdana,Arial,Helvetica" size="-1">00060 Discharge(Mean)</font></label></td>
<td nowrap="nowrap"><font face="Verdana,Arial,Helvetica" size="-1">2012-04-11 </font></td>
<td nowrap="nowrap"><font face="Verdana,Arial,Helvetica" size="-1">2021-05-28</font></td>
</tr>


We may want to use checkbox_00060 as the id and then pick the next td after that....

In [10]:
# find "Discharge" in row_sum
'discharge' in rows_sum.td.get_text().lower()


AttributeError: ResultSet object has no attribute 'td'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
# this loop will pull data for both the 
# list of gages that we already pulled data for
# for i, row in dt:
    
    # get list of USGS gages
    
    # for gage in USGS_list:
        # if gage is not in list of already pulled data
            # get CSV file of daily summary of water flow
            # save CSV file
            # add gage to list of already pulled data
            
        

In the notebook after that:
    - build models (using Lasso Regression, we believe)
    - set threshold for accuracy in predicting flow
    - throw out models that do not meet that accuracy
    - throw out features in models that are no longer needed
    - do other prep for putting models into production (test in real time and have accuracy feedback)